In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import pandas as pd
pd.options.display.max_rows = None
pd.options.display.max_colwidth = 1000


In [1]:
from loaders.cognicity_loader import CognicityLoader
from learners.perceptron_learner import PerceptronLearner
from learners.svm_learner import SvmLearner

# import chennai_config
# config = chennai_config.config

import jakarta_config
config = jakarta_config.config

# import chennai_only_pics_config
# config = chennai_only_pics_config.config

RERUN = True

from image_recognition.goog_recog import GoogleLabeler
#goog_learner = PerceptronLearner(config, CognicityLoader, GoogleLabeler)
goog_learner = SvmLearner(config, CognicityLoader, GoogleLabeler)

from image_recognition.aws_recog import AwsLabeler
#aws_learner = PerceptronLearner(config, CognicityLoader, AwsLabeler)
aws_learner = SvmLearner(config, CognicityLoader, AwsLabeler)

2019-08-22 22:30:24,196 - DEBUG - CognicityLoader constructed
2019-08-22 22:30:24,199 - DEBUG - GoogleLabeler constructed
2019-08-22 22:30:24,200 - DEBUG - CognicityLoader constructed
2019-08-22 22:30:24,200 - DEBUG - GoogleLabeler constructed
2019-08-22 22:30:24,255 - DEBUG - CognicityLoader constructed
2019-08-22 22:30:24,256 - DEBUG - AwsLabeler constructed
2019-08-22 22:30:24,256 - DEBUG - CognicityLoader constructed
2019-08-22 22:30:24,257 - DEBUG - AwsLabeler constructed


In [ ]:
import random
validation_set = set(random.sample(config["flood_pkeys"], 5))
validation_set = validation_set.union(set(random.sample(config["no_flood_pkeys"], 5)))
len(validation_set)

In [ ]:
th, th0 = goog_learner.run_learner("goog_separator.p", rerun=True, validation_keys=validation_set, params={"T":3000, "print":True})
goog_learner.t_data_w_pkey[0,:]
val = goog_learner.val_data_w_pkey[0,:]
goog_learner.val_labels

In [ ]:
goog_learner.val_sd
goog_learner.t_sd.shape

In [ ]:
th, th0 = aws_learner.run_learner("aws_separator.p", rerun=RERUN, validation_keys=validation_set, params={"T":3000, "print":True})

In [ ]:
th.shape
aws_learner.t_sd.shape

In [ ]:
labs = aws_learner.labeler
len(labs.features)

In [ ]:
aws_labeler = aws_learner.labeler
a = aws_labeler.config["flood_pkeys"]
b = aws_labeler.config["no_flood_pkeys"]
c = a.union(b)
max(c)

In [ ]:
from nlp.bow_labeler import BowLabeler
from learners.svm_learner import SvmLearner
#bow_learner = PerceptronLearner(config, CognicityLoader, BowLabeler)
bow_learner = SvmLearner(config, CognicityLoader, BowLabeler)
th, th0 = bow_learner.run_learner("bow_separator.p", rerun=RERUN, validation_keys=validation_set, params={"T":1000, "print":True})

from sklearn.model_selection import cross_val_score
data = bow_learner.t_data_w_pkey[1:,:].T
labels = bow_learner.t_labels[0,:]

s = bow_learner.cross_validate_model(data, labels,k=5)

scores = cross_val_score(bow_learner.clf, data, labels, cv=10)

scores.mean(), scores.std()*2

In [ ]:
s

# chennai svm: 
# not random shuffle:
# (0.761950146627566, 0.020836769409597528)
# rand shuffle: 
# (0.725, 0.075)

In [ ]:
from flood_depth.flood_labeler import IdentityLabeler

In [2]:
from learners.identity_learner import IdentityLearner
from learners.identity_learner import IdentityLearner
from flood_depth.flood_labeler import IdentityLabeler
from image_recognition.goog_recog import GoogleLabeler
from nlp.bow_labeler import BowLabeler

identity_learner = IdentityLearner(config, CognicityLoader, IdentityLabeler)


from learners.ensemble_learner import EnsembleLearner
import random

fin = []
results= dict()
for i in range(5, 15):
    
    validation_set = set(random.sample(config["flood_pkeys"], 15))
    validation_set = validation_set.union(set(random.sample(config["no_flood_pkeys"], 15)))
    
    # from sklearn.model_selection import ShuffleSplit
    # rs = ShuffleSplit(n_splits =5, test_size=.10)
    # 
    # 
    aws_learner = SvmLearner(config, CognicityLoader, AwsLabeler)
    th, th0 = aws_learner.run_learner("aws_separator.p", rerun=RERUN, validation_keys=validation_set, params={"T":1000, "print":True})
    
    bow_learner = SvmLearner(config, CognicityLoader, BowLabeler)
    th, th0 = bow_learner.run_learner("bow_separator.p", rerun=RERUN, validation_keys=validation_set, params={"T":400, "print":True})
    
    goog_learner = SvmLearner(config, CognicityLoader, GoogleLabeler)
    th, th0 = goog_learner.run_learner("goog_separator.p", rerun=True, validation_keys=validation_set, params={"T":1000, "print":True})
    
    fh_learner = IdentityLearner(config, CognicityLoader, IdentityLabeler)
    meh = fh_learner.run_learner("iden_separator.p", rerun=RERUN, validation_keys=validation_set, params={"T":400, "print":True})
    
    learners = [aws_learner, goog_learner, bow_learner, fh_learner]
    names = ["aws.p", "goog.p", "bow.p", "fh.p"]
    
    # learners = [aws_learner, goog_learner, bow_learner]
    # names = ["aws.p", "goog.p", "bow.p"]
    
    # aws_learner = SvmLearner(config, CognicityLoader, AwsLabeler)
    # th, th0 = aws_learner.run_learner("aws_separator.p", rerun=RERUN, validation_keys=validation_set, params={"T":1000, "print":True})
    # bow_learner = SvmLearner(config, CognicityLoader, BowLabeler)
    # th, th0 = bow_learner.run_learner("bow_separator.p", rerun=RERUN, validation_keys=validation_set, params={"T":400, "print":True})
    # 
    # learners = [aws_learner, bow_learner]
    # names = ["aws.p", "bow.p"]
    
    fh_learner = IdentityLearner(config, CognicityLoader, IdentityLabeler)
    meh = fh_learner.run_learner("iden_separator.p", rerun=RERUN, validation_keys=validation_set, params={"T":400, "print":True})
    
    
    en_learner = EnsembleLearner(config, names, learners)
    
    
    
    nn_model = en_learner.run_learner("chennai_en.p", rerun=RERUN, validation_keys=validation_set, params={"hidden":i, "print":True, "T":5000})
    import math
    probs = math.e**en_learner.res
    probs
    import numpy as np
    p = probs.data.numpy()
    predicted = np.argmax(p, axis=1)
    val_labs = en_learner.val_labels
    true = np.where(val_labs <0, 0, 1)
    score = np.sum(predicted == true)/(true.shape[0])
    fin.append(score)
    print("Ensemble Score: ", score)
    
    import math
    probs = math.e**en_learner.res
    probs
    
    import numpy as np
    p = probs.data.numpy()
    predicted = np.argmax(p, axis=1)
    val_labs = en_learner.val_labels
    true = np.where(val_labs <0, 0, 1)
    score = np.sum(predicted == true)/(true.shape[0])
    results[i] = (score, validation_set, en_learner, val_labs)
    print(score)
    
    print("COMPARE WITH SVM: ")
    t_data = en_learner.train_matrix
    t_labels = en_learner.t_labels
    val_data = en_learner.val_matrix
    val_labels  = en_learner.val_labels
    
    from sklearn import svm
    clf = svm.SVC(gamma="scale", kernel="rbf", degree=3)
            # sklearn expects rows to be data points, we've gone with columns
    clf.fit(t_data.T, t_labels)
    
    pred = clf.predict(val_data.T)
    
    correct = np.sum(val_labels == pred)
    
    total = val_data.shape[1]
    percent_correct = correct/total
    print("Num Correct " + str(correct) +
                     " Out of " + str(total))
    print("Val score: " + str(percent_correct))
    print("____________")

# mispredicted keys:

print(fin)

2019-08-22 22:30:29,133 - DEBUG - CognicityLoader constructed
2019-08-22 22:30:29,133 - DEBUG - IdentityLabeler constructed
2019-08-22 22:30:29,134 - DEBUG - CognicityLoader constructed
2019-08-22 22:30:29,134 - DEBUG - IdentityLabeler constructed
2019-08-22 22:30:29,308 - DEBUG - CognicityLoader constructed
2019-08-22 22:30:29,309 - DEBUG - AwsLabeler constructed
2019-08-22 22:30:29,310 - DEBUG - CognicityLoader constructed
2019-08-22 22:30:29,310 - DEBUG - AwsLabeler constructed
2019-08-22 22:30:29,311 - DEBUG - logging from: default_jakarta_data/aws_labels_default.p
2019-08-22 22:30:45,383 - INFO - Num Correct 19 Out of 29
2019-08-22 22:30:45,384 - INFO - Val score: 0.6551724137931034
2019-08-22 22:30:53,558 - DEBUG - CognicityLoader constructed
2019-08-22 22:30:53,559 - DEBUG - CognicityLoader constructed
2019-08-22 22:31:27,600 - INFO - Num Correct 21 Out of 29
2019-08-22 22:31:27,601 - INFO - Val score: 0.7241379310344828
2019-08-22 22:31:52,596 - DEBUG - CognicityLoader construc

Epoch: 0
Loss: tensor(0.9030, grad_fn=<NllLossBackward>)
Epoch: 500
Loss: tensor(0.6511, grad_fn=<NllLossBackward>)
Epoch: 1000
Loss: tensor(0.6190, grad_fn=<NllLossBackward>)
Epoch: 1500
Loss: tensor(0.5938, grad_fn=<NllLossBackward>)
Epoch: 2000
Loss: tensor(0.5715, grad_fn=<NllLossBackward>)
Epoch: 2500
Loss: tensor(0.5418, grad_fn=<NllLossBackward>)
Epoch: 3000
Loss: tensor(0.5220, grad_fn=<NllLossBackward>)
Epoch: 3500
Loss: tensor(0.5052, grad_fn=<NllLossBackward>)
Epoch: 4000
Loss: tensor(0.4907, grad_fn=<NllLossBackward>)
Epoch: 4500
Loss: tensor(0.4784, grad_fn=<NllLossBackward>)
Epoch: 5000
Loss: tensor(0.4679, grad_fn=<NllLossBackward>)
Epoch: 5500
Loss: tensor(0.4591, grad_fn=<NllLossBackward>)
Epoch: 6000
Loss: tensor(0.4517, grad_fn=<NllLossBackward>)
Epoch: 6500
Loss: tensor(0.4454, grad_fn=<NllLossBackward>)
Epoch: 7000
Loss: tensor(0.4402, grad_fn=<NllLossBackward>)
Epoch: 7500
Loss: tensor(0.4358, grad_fn=<NllLossBackward>)
Epoch: 8000
Loss: tensor(0.4320, grad_fn=<Nl

2019-08-22 22:32:39,764 - DEBUG - dumping model to: default_jakarta_data/chennai_en.p
2019-08-22 22:32:39,828 - DEBUG - CognicityLoader constructed
2019-08-22 22:32:39,829 - DEBUG - AwsLabeler constructed
2019-08-22 22:32:39,830 - DEBUG - CognicityLoader constructed
2019-08-22 22:32:39,830 - DEBUG - AwsLabeler constructed
2019-08-22 22:32:39,831 - DEBUG - logging from: default_jakarta_data/aws_labels_default.p


Ensemble Score:  0.7333333333333333
0.7333333333333333
COMPARE WITH SVM: 
Num Correct 22 Out of 30
Val score: 0.7333333333333333
____________


2019-08-22 22:32:55,979 - INFO - Num Correct 14 Out of 29
2019-08-22 22:32:55,980 - INFO - Val score: 0.4827586206896552
2019-08-22 22:33:03,945 - DEBUG - CognicityLoader constructed
2019-08-22 22:33:03,945 - DEBUG - CognicityLoader constructed
2019-08-22 22:33:38,482 - INFO - Num Correct 17 Out of 28
2019-08-22 22:33:38,483 - INFO - Val score: 0.6071428571428571
2019-08-22 22:34:03,981 - DEBUG - CognicityLoader constructed
2019-08-22 22:34:03,981 - DEBUG - GoogleLabeler constructed
2019-08-22 22:34:03,982 - DEBUG - CognicityLoader constructed
2019-08-22 22:34:03,983 - DEBUG - GoogleLabeler constructed
2019-08-22 22:34:12,366 - INFO - Num Correct 19 Out of 29
2019-08-22 22:34:12,367 - INFO - Val score: 0.6551724137931034
2019-08-22 22:34:16,110 - DEBUG - CognicityLoader constructed
2019-08-22 22:34:16,111 - DEBUG - IdentityLabeler constructed
2019-08-22 22:34:16,111 - DEBUG - CognicityLoader constructed
2019-08-22 22:34:16,112 - DEBUG - IdentityLabeler constructed
2019-08-22 22:34:19,9

...

Early stopping at epoch: 40
Ensemble Score:  0.5
0.5
COMPARE WITH SVM: 
Num Correct 23 Out of 30
Val score: 0.7666666666666667
____________


2019-08-22 22:51:56,372 - INFO - Num Correct 19 Out of 30
2019-08-22 22:51:56,373 - INFO - Val score: 0.6333333333333333
2019-08-22 22:52:04,338 - DEBUG - CognicityLoader constructed
2019-08-22 22:52:04,339 - DEBUG - CognicityLoader constructed
2019-08-22 22:52:41,037 - INFO - Num Correct 23 Out of 30
2019-08-22 22:52:41,039 - INFO - Val score: 0.7666666666666667
2019-08-22 22:53:05,850 - DEBUG - CognicityLoader constructed
2019-08-22 22:53:05,855 - DEBUG - GoogleLabeler constructed
2019-08-22 22:53:05,855 - DEBUG - CognicityLoader constructed
2019-08-22 22:53:05,856 - DEBUG - GoogleLabeler constructed
2019-08-22 22:53:14,921 - INFO - Num Correct 20 Out of 30
2019-08-22 22:53:14,922 - INFO - Val score: 0.6666666666666666
2019-08-22 22:53:18,391 - DEBUG - CognicityLoader constructed
2019-08-22 22:53:18,392 - DEBUG - IdentityLabeler constructed
2019-08-22 22:53:18,393 - DEBUG - CognicityLoader constructed
2019-08-22 22:53:18,394 - DEBUG - IdentityLabeler constructed
2019-08-22 22:53:22,5

Epoch: 0
Loss: tensor(0.6361, grad_fn=<NllLossBackward>)
Epoch: 500
Loss: tensor(0.5998, grad_fn=<NllLossBackward>)
Epoch: 1000
Loss: tensor(0.5697, grad_fn=<NllLossBackward>)
Epoch: 1500
Loss: tensor(0.5181, grad_fn=<NllLossBackward>)
Epoch: 2000
Loss: tensor(0.4550, grad_fn=<NllLossBackward>)
Epoch: 2500
Loss: tensor(0.4043, grad_fn=<NllLossBackward>)
Epoch: 3000
Loss: tensor(0.3633, grad_fn=<NllLossBackward>)
Epoch: 3500
Loss: tensor(0.3301, grad_fn=<NllLossBackward>)
Epoch: 4000
Loss: tensor(0.3036, grad_fn=<NllLossBackward>)
Epoch: 4500
Loss: tensor(0.2829, grad_fn=<NllLossBackward>)
Epoch: 5000
Loss: tensor(0.2669, grad_fn=<NllLossBackward>)
Epoch: 5500
Loss: tensor(0.2545, grad_fn=<NllLossBackward>)
Epoch: 6000
Loss: tensor(0.2451, grad_fn=<NllLossBackward>)
Epoch: 6500
Loss: tensor(0.2378, grad_fn=<NllLossBackward>)
Epoch: 7000
Loss: tensor(0.2322, grad_fn=<NllLossBackward>)
Epoch: 7500
Loss: tensor(0.2278, grad_fn=<NllLossBackward>)
Epoch: 8000
Loss: tensor(0.2243, grad_fn=<Nl

2019-08-22 22:53:53,479 - DEBUG - dumping model to: default_jakarta_data/chennai_en.p


Ensemble Score:  0.6666666666666666
0.6666666666666666
COMPARE WITH SVM: 
Num Correct 19 Out of 30
Val score: 0.6333333333333333
____________
[0.7333333333333333, 0.5333333333333333, 0.6666666666666666, 0.7333333333333333, 0.6333333333333333, 0.6333333333333333, 0.7, 0.6, 0.5, 0.6666666666666666]


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit

# make sure to shuffle the data!
cv = ShuffleSplit(n_splits=k, test_size=.10)
scores = cross_val_score(self.clf, data, labels, cv=cv)
self.scores = scores
return (scores.mean(), scores.std())


In [ ]:
import torch
# print(en_learner.val_matrix)
# print(en_learner.val_matrix[-1,:])
# print(predicted)
# np.sum(np.sum(en_learner.train_matrix[1:-1,:], axis=0) == 0)
val_matrix = en_learner.val_matrix
label_matrix = en_learner.val_labels


into_sk = val_matrix.T
print(into_sk.shape)
into_torch = torch.from_numpy(into_sk).float()

en_learner.predict(en_learner.val_matrix.T)

# from simple_nn import Simple_nn
# 
# nn = Simple_nn(into_sk.shape[1], 10)
# nn.forward(into_torch)
# 
# toy = EnsembleLearner(config, [3, 3], [3, 3])
# 
# 
# print(into_sk.shape)
# print(into_torch.size)
# toy.fit(into_sk, label_matrix)

In [ ]:
t_data = en_learner.train_matrix
t_labels = en_learner.t_labels
val_data = en_learner.val_matrix
val_labels  = en_learner.val_labels

from sklearn import svm
clf = svm.SVC(gamma="scale", kernel="poly", degree=3)
        # sklearn expects rows to be data points, we've gone with columns
clf.fit(t_data.T, t_labels)

pred = clf.predict(val_data.T)

correct = np.sum(val_labels == pred)

total = val_data.shape[1]
percent_correct = correct/total
print("Num Correct " + str(correct) +
                 " Out of " + str(total))
print("Val score: " + str(percent_correct))
print(pred)
print(val_labels)
t_data[:,2:5]

len(config["flood_pkeys"])/len(config["all_pkeys"])

In [ ]:
wrong_pkeys = goog_learner.val_data_w_pkey[0, (predicted != true)[0,:]]

import pandas as pd
start_known_flood = "'2017-11-01 00:00:35.630000-04:00'" 
end_known_flood = "'2019-11-07 00:00:35.630000-04:00'"

chennai_all_data = pd.read_sql_query('''
    SELECT pkey, created_at, text, disaster_type, report_data, tags, image_url FROM riskmap.all_reports 
''', params={"start_date": start_known_flood, "end_date": end_known_flood}, con=config["database_engine"], index_col="pkey")
pd.options.display.max_rows = None
pd.options.display.max_colwidth = 1000

chennai_all_data.loc[pd.Index(wrong_pkeys)]

In [ ]:
p

In [ ]:
predicted

In [ ]:
a = list(range(1, 7))
a.extend([7 for i in range(10)])
a
diff = a[-1] - a[0]
diff

In [ ]:
a.pop(0)
a

In [ ]:
predicted

validation_set

In [ ]:
import numpy as np
import pandas as pd
a = np.array([[1, 2, 4, 65, 75],
         [113, 222, 4444, 65455, 7777]])

b = np.array([[1, 2, 4, 65, 70, 75]])

c = np.vstack((b, np.zeros((1, b.shape[1]))))

b_df = pd.Index(b.T)

c_df = pd.DataFrame(c.T, columns=["pkey", "data"])

a_df = pd.DataFrame(a.T, columns=["pkey", "data"] )

c_df["pkey"] == a_df["pkey"]

In [ ]:
import numpy as np
import pandas as pd
a = np.array([[1, 2, 4, 65, 75],
         [113, 222, 4444, 65455, 7777]])

labs = np.array([[1, 1, 1, -1, -1]])
b = np.array([[1, 2, 4, 65, 70, 75]])

c = np.vstack((b, np.zeros((1, b.shape[1]))))

b_df = pd.Index(b.T)

c_df = pd.DataFrame(c.T, columns=["pkey", "data"]).set_index("pkey", drop=False)

a_df = pd.DataFrame(a.T, columns=["pkey", "data"] )
# a_df.rename_axis( "pkey")
a_df.set_index("pkey", inplace=True, drop=False)
a_df

# c_df.loc[a_df.index] = a_df["data"]
c_df.loc[a_df.index] = a_df.loc[a_df.index]
eh = c_df.to_numpy()
print(eh)
eh[1:, :]

In [ ]:
import chennai_config
config = chennai_config.config
def _fill_no_data_spots(data_list, label_list):
    all_pkeys = set([1, 2, 4, 65, 70, 75]) # config["all_pkeys"]
    res = pd.DataFrame(all_pkeys, columns=["pkey"]).set_index("pkey", drop=False).sort_index()
    for i, (data, labels) in enumerate(zip(data_list, label_list)):
        add_df = pd.DataFrame(data.T, columns=["pkey", i]).set_index("pkey")
        res = res.join(add_df).fillna(0)
        # res.loc[add_df.index][i] = add_df.loc[a_df.index][i]
        print(add_df.loc[a_df.index][i])
        
    flood = pd.Index([1, 2, 4])
    no_flood = pd.Index([65, 70, 75])
    # add in the label data as the last row
    res.loc[flood, "label"] = 1
    res.loc[no_flood, "label"] = -1
    ahhh = res.to_numpy().T
    # TODO there's probably some issue with floating point equality on the pkeys here... 
    return ahhh

_fill_no_data_spots([a, a], [labs, labs] )